In [ ]:
import json
from datetime import datetime

# 讀取 JSON 檔案
with open(r'C:\Users\student\Desktop\disney\history_test.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 新增一個鍵值對，儲存星期幾
for record in data:
    datetime_str = record['datetime']
    dt = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M')
    record['weekday'] = dt.strftime('%A')

# 輸出結果
for record in data:
    print(f"日期: {record['datetime']}, 星期: {record['weekday']}")

# 如果需要將結果保存到新的 JSON 檔案中
with open(r'C:\Users\student\Desktop\disney\save_time.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [ ]:
# 導入必要的庫
import pandas as pd
import json
from datetime import timedelta

# 讀取 JSON 檔案
file_path = '/Users/chianlee/Desktop/disney/testing_data_sea.json'
file_path_names = '/Users/chianlee/Desktop/disney/sea_namelist.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with open(file_path_names, 'r', encoding='utf-8') as file:
    names = json.load(file)

In [ ]:
# 轉換為 DataFrame
df_data = pd.DataFrame(data)
df_names = pd.DataFrame(names)

# 刪除 'FacilityName' 欄位
df_data = df_data.drop(columns=['FacilityName'])

# 刪除 'StandbyTime' 為 null 值的行
df_data = df_data[df_data['StandbyTime'].notna()]

# 顯示處理後的數據
df_data.head(5)

In [ ]:
# 解析日期時間欄位
df_cleaned['UpdateTime'] = pd.to_datetime(df_cleaned['UpdateTime'], format='%H:%M')
df_cleaned['datetime'] = pd.to_datetime(df_cleaned['datetime'])

# 提取日期並應用於 UpdateTime
df_cleaned['UpdateTime'] = df_cleaned.apply(
    lambda row: row['UpdateTime'].replace(year=row['datetime'].year, month=row['datetime'].month, day=row['datetime'].day),
    axis=1
)

# 計算 UpdateTime 和 datetime 之間的時間差（以小時為單位）
df_cleaned['time_difference'] = (df_cleaned['datetime'] - df_cleaned['UpdateTime'])

# 刪除時間差超過1小時的資料
df_filtered = df_cleaned[df_cleaned['time_difference'] <= timedelta(hours=1)]

# 刪除多餘的時間差欄位
df_filtered = df_filtered.drop(columns=['time_difference'])

# 顯示處理後的數據
df_filtered.head()

In [ ]:
df_filtered.describe()

In [ ]:
df_filtered.shape

In [ ]:
df_filtered.isnull().sum()

In [ ]:
df_filtered.info()